In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q langchain langchain-community pypdf
! pip install -q chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

from pathlib import Path

import json

In [5]:
DATA_PATH = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_image_600.json")
CHROMA_QA_DIR = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_qa_600")

CHROMA_QA_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
questions = json.loads(DATA_PATH.read_text(encoding="utf-8"))

def build_text(q: dict) -> str:
    """Ghép câu hỏi + phương án + đáp án thành 1 chuỗi để embed."""
    lines = []
    lines.append(q["question"])
    lines.append("Các phương án:")

    for opt in q["options"]:
        opt_text = opt["text"] if isinstance(opt, dict) else opt
        lines.append(f"- {opt_text}")

    lines.append(f"Đáp án đúng: {q['answer']}")
    return "\n".join(lines)

qa_docs: list[Document] = []
for i, q in enumerate(questions, start=1):
    q_id = q.get("id") or f"Q{i:03d}"

    qa_docs.append(
        Document(
            page_content=build_text(q),
            metadata={
                "id": q_id,
                "source_doc": q['source_doc'],
                "question": q["question"],
                "answer": q["answer"],
            },
        )
    )

print(f"Tổng số câu hỏi: {len(qa_docs)}")


Tổng số câu hỏi: 599


In [7]:
qa_docs[0]

Document(metadata={'id': 'Q001', 'source_doc': 'Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf', 'question': 'Câu 1. Phần của đường bộ được sử dụng cho phương tiện giao thông đường bộ đi lại là gì?', 'answer': '2. Phần đường xe chạy.'}, page_content='Câu 1. Phần của đường bộ được sử dụng cho phương tiện giao thông đường bộ đi lại là gì?\nCác phương án:\n- 1. Phần mặt đường và lề đường.\n- 2. Phần đường xe chạy.\n- 3. Phần đường xe cơ giới.\nĐáp án đúng: 2. Phần đường xe chạy.')

In [11]:
txt_path = "/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_chunks.txt"

with open(txt_path, "w", encoding="utf-8") as f:
    for i, doc in enumerate(qa_docs):
        f.write(f"===== CHUNK {i} =====\n")
        f.write(f"id: {doc.metadata['id']}\n")
        f.write(doc.page_content)
        f.write("\n\n\n")

print("Đã lưu file TXT tại:", txt_path)


Đã lưu file TXT tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_chunks.txt


In [ ]:
# ============ 3) KHỞI TẠO EMBEDDING MODEL (TEXT) ============

embeddings = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
# ============ 4) BUILD CHROMA + EMBED TEXT ============

qa_vectorstore = Chroma.from_documents(
    documents=qa_docs,
    embedding=embeddings,
    persist_directory=str(CHROMA_QA_DIR),
)

qa_vectorstore.persist()
print("Đã build xong Chroma cho 600 câu tại:", CHROMA_QA_DIR)

Đã build xong Chroma cho 600 câu tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_qa_600


In [ ]:
retriever = qa_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

In [ ]:
# Ví dụ truy vấn
query = "làn đường là gì ?"

docs = retriever.invoke(query)  # LangChain >= 0.3 dùng .invoke()

print(f"Tìm được {len(docs)} chunk\n")
for i, d in enumerate(docs, start=1):
    print(f"===== KẾT QUẢ {i} =====")
    print("Source:", d.metadata.get("source_file"))
    print("Page:", d.metadata.get("page"))
    print("Nội dung:")
    print(d.page_content[:500], "...")
    print()

Tìm được 5 chunk

===== KẾT QUẢ 1 =====
Source: None
Page: None
Nội dung:
Câu 2. Làn đường là gì?
Các phương án:
- 1. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, sử dụng cho xe chạy.
- 2. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, có đủ chiều rộng cho xe chạy an toàn.
- 3. Là đường cho xe ô tô chạy, dừng, đỗ an toàn.
Đáp án đúng: 2. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, có đủ chiều rộng cho xe chạy an toàn. ...

===== KẾT QUẢ 2 =====
Source: None
Page: None
Nội dung:
Câu 2. Làn đường là gì?
Các phương án:
- 1. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, sử dụng cho xe chạy.
- 2. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, có đủ chiều rộng cho xe chạy an toàn.
- 3. Là đường cho xe ô tô chạy, dừng, đỗ an toàn.
Đáp án đúng: 2. Là một phần của phần đường xe chạy được chia theo chiều dọc của đường, có đủ chiều rộng cho xe chạy an toàn. ...

===== KẾT 